### 装饰器

装饰器：用一个函数装饰另一个函数并为其提供额外的能力

装饰器本身就是函数，它的参数是被装饰的函数，返回值是一个带有装饰功能的函数。

In [1]:
import random
import time

def download(filename):
    """下载文件"""
    print(f"开始下载 {filename} ...")
    time.sleep(random.random() * 6)
    print(f"{filename} 下载完成！")

def upload(filename):
    """上传文件"""
    print(f"开始上传 {filename} ...")
    time.sleep(random.random() * 8)
    print(f"{filename} 上传完成！")

download('MySQL从删库到跑路.avi')
upload('Python从入门到住院.pdf')

开始下载 MySQL从删库到跑路.avi ...
MySQL从删库到跑路.avi 下载完成！
开始上传 Python从入门到住院.pdf ...
Python从入门到住院.pdf 上传完成！


在函数开始执行的时候记录一个时间，然后再调用结束后记录一个时间，两个时间相减就可以计算出下载或上传的时间

In [2]:
start = time.time()
download('MySQL从删库到跑路.avi')
end = time.time()
print(f"下载耗时：{end - start:.2f} 秒")

start = time.time()
upload('Python从入门到住院.pdf')
end = time.time()
print(f"下载耗时：{end - start:.2f} 秒")

开始下载 MySQL从删库到跑路.avi ...
MySQL从删库到跑路.avi 下载完成！
下载耗时：2.64 秒
开始上传 Python从入门到住院.pdf ...
Python从入门到住院.pdf 上传完成！
下载耗时：6.90 秒


以上的写法都有重复，为了解决代码重复的问题，我们可以通过装饰器语法，可以把和原来业务（上传下载）没有关系计时功能的代码封装到一个函数中，如果两个函数需要记录时间，可以直接把装饰器作用上去。

In [4]:
def record_time(func):
    """装饰器：记录函数执行时间"""
    def wrapper(*args, **kwargs):
        """包装函数"""
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} 执行耗时：{end - start:.2f} 秒")
        return result
    return wrapper

download = record_time(download)
upload = record_time(upload)
download('MySQL从删库到跑路.avi')
upload('Python从入门到住院.pdf')

开始下载 MySQL从删库到跑路.avi ...
MySQL从删库到跑路.avi 下载完成！
download 执行耗时：2.49 秒
开始上传 Python从入门到住院.pdf ...
Python从入门到住院.pdf 上传完成！
upload 执行耗时：6.75 秒


还有更简单的语法糖，就是 `@装饰器函数`

In [5]:
def record_time(func):
    """装饰器：记录函数执行时间"""
    def wrapper(*args, **kwargs):
        """包装函数"""
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} 执行耗时：{end - start:.2f} 秒")
        return result
    return wrapper

@record_time
def download(filename):
    """下载文件"""
    print(f"开始下载 {filename} ...")
    time.sleep(random.random() * 6)
    print(f"{filename} 下载完成！")

@record_time
def upload(filename):
    """上传文件"""
    print(f"开始上传 {filename} ...")
    time.sleep(random.random() * 8)
    print(f"{filename} 上传完成！")

download('MySQL从删库到跑路.avi')
upload('Python从入门到住院.pdf')

开始下载 MySQL从删库到跑路.avi ...
MySQL从删库到跑路.avi 下载完成！
download 执行耗时：1.50 秒
开始上传 Python从入门到住院.pdf ...
Python从入门到住院.pdf 上传完成！
upload 执行耗时：3.74 秒


如果在某些地方要去掉装饰器，就要做额外的工作，可以通过被装饰函数的__wrapped__属性获得被装饰之前的函数

In [8]:
from functools import wraps

def record_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'{func.__name__}执行时间: {end - start:.2f}秒')
        return result

    return wrapper

@record_time
def download(filename):
    print(f'开始下载{filename}.')
    time.sleep(random.random() * 6)
    print(f'{filename}下载完成.')


@record_time
def upload(filename):
    print(f'开始上传{filename}.')
    time.sleep(random.random() * 8)
    print(f'{filename}上传完成.')
    

# 调用装饰后的函数会记录执行时间
download('MySQL从删库到跑路.avi')
upload('Python从入门到住院.pdf')
# 取消装饰器的作用不记录执行时间
download.__wrapped__('MySQL必知必会.pdf')
upload.__wrapped__('Python从新手到大师.pdf')

开始下载MySQL从删库到跑路.avi.
MySQL从删库到跑路.avi下载完成.
download执行时间: 2.78秒
开始上传Python从入门到住院.pdf.
Python从入门到住院.pdf上传完成.
upload执行时间: 6.45秒
开始下载MySQL必知必会.pdf.
MySQL必知必会.pdf下载完成.
开始上传Python从新手到大师.pdf.
Python从新手到大师.pdf上传完成.


### 递归调用

函数自己调用自己称为递归调用，那么递归调用有什么用处呢？

比如，我们可以使用递归调用的方式来写一个求阶乘的函数，代码如下所示。

In [10]:
def fac(num):
    if num in (0, 1):
        return 1
    return num * fac(num - 1)

# 递归调用函数入栈
# 5 * fac(4)
# 5 * (4 * fac(3))
# 5 * (4 * (3 * fac(2)))
# 5 * (4 * (3 * (2 * fac(1))))
# 停止递归函数出栈
# 5 * (4 * (3 * (2 * 1)))
# 5 * (4 * (3 * 2))
# 5 * (4 * 6)
# 5 * 24
# 120
print(fac(5))    # 120

120


注意，函数调用会通过内存中称为“栈”（stack）的数据结构来保存当前代码的执行现场，函数调用结束后会通过这个栈结构恢复之前的执行现场。

栈是一种先进后出的数据结构，这也就意味着最早入栈的函数最后才会返回，而最后入栈的函数会最先返回。

例如调用一个名为a的函数，函数a的执行体中又调用了函数b，函数b的执行体中又调用了函数c，那么最先入栈的函数是a，最先出栈的函数是c。

每进入一个函数调用，栈就会增加一层栈帧（stack frame），栈帧就是我们刚才提到的保存当前代码执行现场的结构；每当函数调用结束后，栈就会减少一层栈帧。通

常，内存中的栈空间很小，因此递归调用的次数如果太多，会导致栈溢出（stack overflow），所以递归调用一定要确保能够快速收敛。我们可以尝试执行fac(5000)，看看是不是会提示RecursionError错误，错误消息为：maximum recursion depth exceeded in comparison（超出最大递归深度），其实就是发生了栈溢出。

而递归函数有可能会超出栈的范围而溢出

In [11]:
def fib1(n):
    if n in (1, 2):
        return 1
    return fib1(n - 1) + fib1(n - 2)


for i in range(1, 51):
    print(i, fib1(i))

1 1
2 1
3 2
4 3
5 5
6 8
7 13
8 21
9 34
10 55
11 89
12 144
13 233
14 377
15 610
16 987
17 1597
18 2584
19 4181
20 6765
21 10946
22 17711
23 28657
24 46368
25 75025
26 121393
27 196418
28 317811
29 514229
30 832040
31 1346269
32 2178309
33 3524578
34 5702887
35 9227465
36 14930352


KeyboardInterrupt: 

使用 `@lru_cache` 可以优化该递归代码，它可以缓存该函数的执行结果从而避免在递归调用中产生大量的重复运算。

In [ ]:
from functools import lru_cache

"""
lru_cache函数式一个带参数的装饰器，所以后面要有圆括号
lru_cache函数有一个非常重要的参数叫maxsize，它可以用来定义缓存空间的大小，默认值是128。
"""
@lru_cache()
def fib1(n):
    if n in (1, 2):
        return 1
    return fib1(n - 1) + fib1(n - 2)


for i in range(1, 51):
    print(i, fib1(i))

### 总结

装饰器是 Python 语言中的特色语法，可以通过装饰器来增强现有的函数，这是一种非常有用的编程技巧。另一方面，通过函数递归调用，可以在代码层面将一些复杂的问题简单化，但是递归调用一定要注意收敛条件和递归公式，找到递归公式才有机会使用递归调用，而收敛条件则确保了递归调用能停下来。函数调用通过内存中的栈空间来保存现场和恢复现场，栈空间通常都很小，所以递归如果不能迅速收敛，很可能会引发栈溢出错误，从而导致程序的崩溃。